# Notebook for running Serial CTE on ComCam data, using the EPER method.

Initially written 15 Nov 2019 by Craig Lage.
Not based on DM code.

In [1]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import astropy.io.fits as pf
import sys, glob

In [2]:
REPO_DIR = '/project/shared/comCam/'
OUTPUT_DIR = '/home/cslage/ComCam/20191113/'
raftName = 'R22'

In [3]:
# First check the exposure times
filedir = REPO_DIR+'raw/20191113/'
files = np.sort(glob.glob(filedir+'CC_C_20191113_000???/CC_C_20191113_000???_R22_S00.fits'))
numFiles = len(files)
print(numFiles)
# Choosing a 6 second exposure - about 75% full well.
for file in files[77:78]:
    hdulist = pf.open(file, mode='readonly', do_not_scale_image_data=True)
        
    phdr=hdulist[0].header
    filename = phdr['FILENAME']
    exptime = phdr['EXPTIME']
    imgtype = phdr['IMGTYPE'] 
    print("%s\t%s\t%f"%(filename, imgtype, exptime))

279
CC_C_20191113_000078_R22_S00.fits	EXPOSE	6.000000


In [4]:
# Process just this one exposure time, all nine CCDs
filedir = REPO_DIR+'raw/20191113/'
files = np.sort(glob.glob(filedir+'CC_C_20191113_000078/CC_C_20191113_000078_R22_S??.fits'))
numFiles = len(files)
print(numFiles)

for file in files:#[77:78]:
    hdulist = pf.open(file, mode='readonly', do_not_scale_image_data=True)
        
    phdr=hdulist[0].header
    filename = phdr['FILENAME']
    exptime = phdr['EXPTIME']
    imgtype = phdr['IMGTYPE'] 
    print("%s\t%s\t%f"%(filename, imgtype, exptime))

9
CC_C_20191113_000078_R22_S00.fits	EXPOSE	6.000000
CC_C_20191113_000078_R22_S01.fits	EXPOSE	6.000000
CC_C_20191113_000078_R22_S02.fits	EXPOSE	6.000000
CC_C_20191113_000078_R22_S10.fits	EXPOSE	6.000000
CC_C_20191113_000078_R22_S11.fits	EXPOSE	6.000000
CC_C_20191113_000078_R22_S12.fits	EXPOSE	6.000000
CC_C_20191113_000078_R22_S20.fits	EXPOSE	6.000000
CC_C_20191113_000078_R22_S21.fits	EXPOSE	6.000000
CC_C_20191113_000078_R22_S22.fits	EXPOSE	6.000000


In [5]:
# These are for the ITL sensor, idenifying the end of the imaging region and the overscan region.
xstart = 505
xstop = 542
ov_start = 512
# Run most of the rows, but stay away from the edges.
ystart = 200
ystop = 1800
xaxis = np.linspace(xstart,xstop-1,xstop-xstart)

pdf = PdfPages(OUTPUT_DIR+'plots/ComCam_Serial_CTI_20191113000078.pdf')
for file in files:
    print(file)
    name = file[74:77]
    det = file[82:85]
    print(name, det)
    hdulist = pf.open(file, mode='readonly', do_not_scale_image_data=True)
    fig = plt.figure(figsize=(15,15))
    plt.subplots_adjust(hspace = 0.5, wspace = 0.5)
    plt.suptitle("Serial CTE by segment - ITL ComCam, R22_%s\n"%det, fontsize=24)
    for segment in range(16):
            #try:
            plt.subplot(4,4,segment+1)
            imhdr=hdulist[segment+1].header
            extname = imhdr['EXTNAME']
            plt.title("CTE - %s"%extname)
            data = np.array(hdulist[segment+1].data + 32768, dtype = np.int32)
            flat_overscan = np.mean(np.array(data[:,xstop-8:xstop]),axis = 1)
            cte_data = ((np.transpose(np.transpose(data) - flat_overscan))[ystart:ystop,:].mean(axis=0))[xstart:xstop]
            cte_std = ((np.transpose(np.transpose(data) - flat_overscan))[ystart:ystop,:].std(axis=0) / np.sqrt(float(ystop-ystart)))[xstart:xstop]
            cti = np.median((np.transpose(np.transpose(data) - flat_overscan))[ystart:ystop,ov_start]\
            / (np.transpose(np.transpose(data) - flat_overscan))[ystart:ystop,ov_start-1]) / ov_start

            plt.scatter(xaxis, cte_data, color = 'blue')
            plt.errorbar(xaxis, cte_data, yerr=cte_std)
            plt.text(xstart + 8, 20000, "CTI = %.2E"%cti, fontsize = 12)
            plt.yscale('log',nonposy='clip')
            plt.ylim(.01, 200000)
            plt.xticks([510,520,530,540])
            plt.xlabel("X (First overscan = 512)")
            plt.ylabel("Signal(ADU)")
            #print("Segment %s done"%extname)
            #except:
            #continue
    pdf.savefig(fig)  # saves the current figure into a pdf page
    plt.close()
    plt.savefig("/home/cslage/WORK/ats/Serial_CTE_%s_15Aug19.pdf"%name)
    plt.clf()
pdf.close()

/project/shared/comCam/raw/20191113/CC_C_20191113_000078/CC_C_20191113_000078_R22_S00.fits
078 S00
/project/shared/comCam/raw/20191113/CC_C_20191113_000078/CC_C_20191113_000078_R22_S01.fits
078 S01
/project/shared/comCam/raw/20191113/CC_C_20191113_000078/CC_C_20191113_000078_R22_S02.fits
078 S02
/project/shared/comCam/raw/20191113/CC_C_20191113_000078/CC_C_20191113_000078_R22_S10.fits
078 S10
/project/shared/comCam/raw/20191113/CC_C_20191113_000078/CC_C_20191113_000078_R22_S11.fits
078 S11
/project/shared/comCam/raw/20191113/CC_C_20191113_000078/CC_C_20191113_000078_R22_S12.fits
078 S12
/project/shared/comCam/raw/20191113/CC_C_20191113_000078/CC_C_20191113_000078_R22_S20.fits
078 S20
/project/shared/comCam/raw/20191113/CC_C_20191113_000078/CC_C_20191113_000078_R22_S21.fits
078 S21
/project/shared/comCam/raw/20191113/CC_C_20191113_000078/CC_C_20191113_000078_R22_S22.fits
078 S22


<Figure size 432x288 with 0 Axes>

In [6]:
# Query the voltage condition header
hdulist = pf.open(files[0], mode='readonly', do_not_scale_image_data=True)

hdr18=hdulist[17].header
for key in hdr18.keys():
    print(key,hdr18[key])



XTENSION BINTABLE
BITPIX 8
NAXIS 2
NAXIS1 0
NAXIS2 0
PCOUNT 0
GCOUNT 1
TFIELDS 0
CHECKSUM IGlqI9joIEjoI9jo
TEMP1 -9.875
TEMP2 -9.8125
TEMP3 -22.5625
TEMP4 -18.4375
TEMP5 -20.1875
TEMP6 -17.6875
GDP 20.0
GD 0
RDP 13.0
RD 0
OGP -2.0
OG 0
ODP 25.0
OD 0
CSGATEP 1.0
CSGATE 0
AP0_RC 15
AP1_RC 15
AP0_GAIN 1
AP1_GAIN 1
AP0_CLMP 0
AP1_CLMP 0
AP0_AF1 0
AP1_AF1 0
SCK_LOWP -5.0
SCK_LOW 0
SCK_HIP 5.0
SCK_HI 0
PCK_LOWP -8.0
PCK_LOW 0
PCK_HIP 2.0
PCK_HI 0
RG_LOWP -2.0
RG_LOW 0
RG_HIP 8.0
RG_HI 0
HVBIAS UNKNOWN
POWER 0.0
ODVB 0.775
ODIB 0.0
ODVA 0.4738
ODVA2 0.47500000000000003
ODIA 0.0
ODVS 0.0
BSSVBS 0.15000000000000002
BSSIBS 0.0
DATASUM 0
